<a href="https://colab.research.google.com/github/MarioAuditore/TDA-for-Travelling-Salesman/blob/main/train_tsp_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Based on "The Transformer Network for the Traveling Salesman Problem"

Xavier Bresson, Thomas Laurent, Feb 2021<br>

Arxiv : https://arxiv.org/pdf/2103.03012.pdf<br>
Talk : https://ipam.wistia.com/medias/0jrweluovs<br>
Slides : https://t.co/ySxGiKtQL5<br>

This code trains the transformer network by reinforcement learning.<br>
Use the beam search code to test the trained network.


In [ ]:
!git clone https://github.com/MarioAuditore/TDA-for-Travelling-Salesman.git

Cloning into 'TDA-for-Travelling-Salesman'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 51 (delta 2), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (51/51), 104.13 MiB | 16.94 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [ ]:
import os

os.chdir('/content/TDA-for-Travelling-Salesman')

In [ ]:
!pip install 'pyconcorde @ git+https://github.com/jvkersch/pyconcorde'

  Cloning https://github.com/jvkersch/pyconcorde to /tmp/pip-install-rl8x1tcz/pyconcorde_601abf22ad3548519d5f7bac8d6efd49
  Running command git clone --filter=blob:none --quiet https://github.com/jvkersch/pyconcorde /tmp/pip-install-rl8x1tcz/pyconcorde_601abf22ad3548519d5f7bac8d6efd49
  Resolved https://github.com/jvkersch/pyconcorde to commit 8a6b193b79ebdf8f07e0b0635722b3b4edbc1560
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.0 MB/s eta 0:00:00
  Created wheel for pyconcorde: filename=pyconcorde-0.1.0-cp310-cp310-linux_x86_64.whl size=2576468 sha256=169d9e414b4f4c115f1c8e06275ea5873f9b95a0c98d9f13ed33f81f28971746
  Stored in directory: /tmp/pip-ephem-wheel-cache-r4ut8iu8/wheels/be/9f/eb/e4f95e06e62f057a045679e4940e536d1b251d1ab4859c6dcc
Successfully built pyconcorde
  Attempting uninstall: tabu

In [ ]:
# ================
# Libs
# ================

import torch
import torch.nn as nn
from tqdm import tqdm


# Models
from tsp_transformer.model import TSP_net, compute_tour_length


import time
import argparse
import os
import datetime


# visualization
%matplotlib inline
# from IPython.display import set_matplotlib_formats, clear_output
# set_matplotlib_formats('png2x','pdf')
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


import networkx as nx
from scipy.spatial.distance import pdist, squareform
from concorde.tsp import TSPSolver # !pip install -e pyconcorde


import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
###################
# Hardware : CPU / GPU(s)
###################

if torch.backends.mps.is_available():
    gpu_id = '0'
    device = torch.device("mps")

elif torch.cuda.is_available():
    gpu_id = '0' # select a single GPU
    # gpu_id = '2,3' # select multiple GPUs
    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)
    device = torch.device("cuda")
    print('GPU name: {:s}, gpu_id: {:s}'.format(torch.cuda.get_device_name(0),gpu_id))

else:
    device = torch.device("cpu")
    gpu_id = -1 # select CPU


print(device)

GPU name: Tesla T4, gpu_id: 0
cuda


In [ ]:
# ================
# Hyper-parameters
# ================

class DotDict(dict):
    def __init__(self, **kwds):
        self.update(kwds)
        self.__dict__ = self

args = DotDict()
args.gpu_id = gpu_id

# TSP problem number of nodes
args.nb_nodes = 10 # TSP20
# args.nb_nodes = 50 # TSP50
# args.nb_nodes = 100 # TSP100

# Transformer parameters
args.dim_emb = 128 # dimension of embeddings in transformer
args.dim_ff = 512 # dimension of feed forward layers
args.dim_input_nodes = 2
args.nb_layers_encoder = 6
args.nb_layers_decoder = 2
args.nb_heads = 8

#
args.nb_epochs = 20 # number of epochs
args.batch_size = 128 # batch size
args.nb_batch_per_epoch = 250 # number of batches to generate on each epoch for training
args.nb_batch_eval = 100 # number of batches to generate on each epoch for evaluation
args.lr = 1e-4 # optimiser lr
args.tol = 1e-3 # model should perform better w.r.t tolerance to be updated
args.batchnorm = True  # if batchnorm=True  than batch norm is used
# args.batchnorm = False # if batchnorm=False than layer norm is used
args.max_len_PE = 1000

print(args)


{'gpu_id': '0', 'nb_nodes': 10, 'dim_emb': 128, 'dim_ff': 512, 'dim_input_nodes': 2, 'nb_layers_encoder': 6, 'nb_layers_decoder': 2, 'nb_heads': 8, 'nb_epochs': 20, 'batch_size': 128, 'nb_batch_per_epoch': 250, 'nb_batch_eval': 100, 'lr': 0.0001, 'tol': 0.001, 'batchnorm': True, 'max_len_PE': 1000}


# Training
## Setup

In [ ]:
###################
# Instantiate a training network and a baseline network
###################
try:
    del model_train # remove existing model
    del model_baseline # remove existing model
except:
    pass

model_train = TSP_net(args.dim_input_nodes,
                      args.dim_emb,
                      args.dim_ff,
                      args.nb_layers_encoder,
                      args.nb_layers_decoder,
                      args.nb_heads,
                      args.max_len_PE,
                      batchnorm=args.batchnorm)

model_baseline = TSP_net(args.dim_input_nodes,
                         args.dim_emb,
                         args.dim_ff,
                         args.nb_layers_encoder,
                         args.nb_layers_decoder,
                         args.nb_heads,
                         args.max_len_PE,
                         batchnorm=args.batchnorm)

if torch.cuda.device_count() > 1:
    print(torch.cuda.device_count() + " cuda devices found, doing parallel training.")
    model_train = nn.DataParallel(model_train)
    model_baseline = nn.DataParallel(model_baseline)

optimizer = torch.optim.Adam(model_train.parameters(), lr = args.lr)

model_train = model_train.to(device)
model_baseline = model_baseline.to(device)
model_baseline.eval()

print(args); print('')

# Logs
os.system("mkdir logs")
time_stamp=datetime.datetime.now().strftime("%y-%m-%d--%H-%M-%S")
file_name = 'logs'+'/'+time_stamp + "-n{}".format(args.nb_nodes) + "-gpu{}".format(args.gpu_id) + ".txt"
file = open(file_name,"w",1)
file.write(time_stamp+'\n\n')
for arg in vars(args):
    file.write(arg)
    hyper_param_val="={}".format(getattr(args, arg))
    file.write(hyper_param_val)
    file.write('\n')
file.write('\n\n')
plot_performance_train = []
plot_performance_baseline = []
all_strings = []
epoch_ckpt = 0
tot_time_ckpt = 0


# # Uncomment these lines to re-start training with saved checkpoint
# ====================================================================
# checkpoint_file = "checkpoint/checkpoint_21-03-01--17-25-00-n50-gpu0.pkl"
# checkpoint = torch.load(checkpoint_file, map_location=device)
# epoch_ckpt = checkpoint['epoch'] + 1
# tot_time_ckpt = checkpoint['tot_time']
# plot_performance_train = checkpoint['plot_performance_train']
# plot_performance_baseline = checkpoint['plot_performance_baseline']
# model_baseline.load_state_dict(checkpoint['model_baseline'])
# model_train.load_state_dict(checkpoint['model_train'])
# optimizer.load_state_dict(checkpoint['optimizer'])
# print('Re-start training with saved checkpoint file={:s}\n  Checkpoint at epoch= {:d} and time={:.3f}min\n'.format(checkpoint_file,epoch_ckpt-1,tot_time_ckpt/60))
# del checkpoint
# ====================================================================



{'gpu_id': '0', 'nb_nodes': 10, 'dim_emb': 128, 'dim_ff': 512, 'dim_input_nodes': 2, 'nb_layers_encoder': 6, 'nb_layers_decoder': 2, 'nb_heads': 8, 'nb_epochs': 20, 'batch_size': 128, 'nb_batch_per_epoch': 250, 'nb_batch_eval': 100, 'lr': 0.0001, 'tol': 0.001, 'batchnorm': True, 'max_len_PE': 1000}



## Test nodes

In [ ]:
###################
# Small test set for quick algorithm comparison
# Note : this can be removed
###################

save_1000tsp = False

test_size = 10

if save_1000tsp:
    x = torch.rand(test_size, args.nb_nodes, args.dim_input_nodes, device='cpu')
    print(x.size(),x[0])
    data_dir = os.path.join("data")

    if not os.path.exists(data_dir):
        os.makedirs(data_dir)

    if args.nb_nodes==20 : torch.save({ 'x': x, }, '{}.pkl'.format(data_dir + "/1000tsp20"))
    if args.nb_nodes==50 : torch.save({ 'x': x, }, '{}.pkl'.format(data_dir + "/1000tsp50"))
    if args.nb_nodes==100 : torch.save({ 'x': x, }, '{}.pkl'.format(data_dir + "/1000tsp100"))

checkpoint = None

if args.nb_nodes==20 : checkpoint = torch.load("data/1000tsp20.pkl")
if args.nb_nodes==50 : checkpoint = torch.load("data/1000tsp50.pkl")
if args.nb_nodes==100 : checkpoint = torch.load("data/1000tsp100.pkl")

if checkpoint is not None:
    x_test_tsp = checkpoint['x'].to(device)
    n = x_test_tsp.size(1)
    print('nb of nodes :',n)
else:
    x_test_tsp = torch.rand(test_size, args.nb_nodes, args.dim_input_nodes, device='cpu')
    n = x_test_tsp.size(1)
    print('nb of nodes :',n)


nb of nodes : 10


## Training loop

In [ ]:
# ==================
# Main training loop
# ==================


start_training_time = time.time()

for epoch in tqdm(range(0, args.nb_epochs)):

    # re-start training with saved checkpoint
    # epoch += epoch_ckpt

    # -------------------------
    # Train model for one epoch
    # -------------------------
    start = time.time()
    model_train.train()

    for step in range(1, args.nb_batch_per_epoch + 1):

        # generate a batch of random TSP instances
        x = torch.rand(args.batch_size, args.nb_nodes, args.dim_input_nodes, device=device) # size(x)=(batch_size, nb_nodes, dim_input_nodes)

        # compute tours for model
        tour_train, sumLogProbOfActions = model_train(x, deterministic=False) # size(tour_train)=(batch_size, nb_nodes), size(sumLogProbOfActions)=(batch_size)

        # compute tours for baseline
        with torch.no_grad():
            tour_baseline, _ = model_baseline(x, deterministic=True)

        # get the lengths of the tours
        L_train = compute_tour_length(x, tour_train) # size(L_train)=(batch_size)
        L_baseline = compute_tour_length(x, tour_baseline) # size(L_baseline)=(batch_size)

        # backprop
        loss = torch.mean((L_train - L_baseline) * sumLogProbOfActions )
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    time_one_epoch = time.time()-start
    time_tot = time.time()-start_training_time + tot_time_ckpt


    # -----------------
    # Evaluate train model and baseline on 10k random TSP instances
    # -----------------
    model_train.eval()
    mean_tour_length_train = 0
    mean_tour_length_baseline = 0

    for step in range(0, args.nb_batch_eval):
        # generate a batch of random tsp instances
        x = torch.rand(args.batch_size, args.nb_nodes, args.dim_input_nodes, device=device)

        # compute tour for model and baseline
        with torch.no_grad():
            tour_train, _ = model_train(x, deterministic=True)
            tour_baseline, _ = model_baseline(x, deterministic=True)

        # get the lengths of the tours
        L_train = compute_tour_length(x, tour_train)
        L_baseline = compute_tour_length(x, tour_baseline)

        # L_tr and L_bl are tensors of shape (batch_size,). Compute the mean tour length
        mean_tour_length_train += L_train.mean().item()
        mean_tour_length_baseline += L_baseline.mean().item()

    mean_tour_length_train =  mean_tour_length_train/ args.nb_batch_eval
    mean_tour_length_baseline =  mean_tour_length_baseline/ args.nb_batch_eval

    # evaluate train model and baseline and update if train model is better
    update_baseline = mean_tour_length_train + args.tol < mean_tour_length_baseline
    if update_baseline:
        model_baseline.load_state_dict(model_train.state_dict())

    # For new baseline compute TSPs for small test set
    with torch.no_grad():
        tour_baseline, _ = model_baseline(x_test_tsp.to(device), deterministic=True)
    mean_tour_length_test = compute_tour_length(x_test_tsp, tour_baseline.to('cpu')).mean().item()

    # For checkpoint
    plot_performance_train.append([(epoch+1), mean_tour_length_train])
    plot_performance_baseline.append([(epoch+1), mean_tour_length_baseline])

    # Compute optimality gap
    if args.nb_nodes==50: gap_train = mean_tour_length_train/5.692 - 1.0
    elif args.nb_nodes==100: gap_train = mean_tour_length_train/7.765 - 1.0
    else: gap_train = -1.0


    # ОНИ БЛЯТЬ ДЕБИЛЫ? ЧТО ТУТ ВООБЩЕ НАХУЙ ПРОСИХОДИТ? КАКОГО ХЕРА ОНИ ВЫШЕ ДЕЛЯТ НА РАНДОМНЫЕ ЧИСЛА В gap_train?????
    # # Print and save in txt file
    # mystring_min = 'Epoch: {:d}, epoch time: {:.3f}min, tot time: {:.3f}day, L_train: {:.3f}, L_base: {:.3f}, L_test: {:.3f}, gap_train(%): {:.3f}, update: {}'.format(
    #     epoch, time_one_epoch/60, time_tot/86400, mean_tour_length_train, mean_tour_length_baseline, mean_tour_length_test, 100*gap_train, update_baseline)
    # print(mystring_min) # Comment if plot display
    # file.write(mystring_min+'\n')

    mystring_min = 'Epoch: {:d}, epoch time: {:.3f} min, tot time: {:.3f}day, L_train: {:.3f}, L_base: {:.3f}, L_test: {:.3f}, gap_train(%): {:.3f}, update: {}'.format(
        epoch, time_one_epoch/60, time_tot/86400, mean_tour_length_train, mean_tour_length_baseline, mean_tour_length_test, 100*gap_train, update_baseline)
    print(mystring_min) # Comment if plot display

    # all_strings.append(mystring_min) # Uncomment if plot display
    # for string in all_strings:
    #     print(string)

    # Saving checkpoint
    checkpoint_dir = os.path.join("checkpoint")
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    torch.save({
        'epoch': epoch,
        'time': time_one_epoch,
        'tot_time': time_tot,
        'loss': loss.item(),
        'TSP_length': [torch.mean(L_train).item(), torch.mean(L_baseline).item(), mean_tour_length_test],
        'plot_performance_train': plot_performance_train,
        'plot_performance_baseline': plot_performance_baseline,
        'mean_tour_length_test': mean_tour_length_test,
        'model_baseline': model_baseline.state_dict(),
        'model_train': model_train.state_dict(),
        'optimizer': optimizer.state_dict(),
        }, '{}.pkl'.format(checkpoint_dir + "/checkpoint_" + time_stamp + "-n{}".format(args.nb_nodes) + "-gpu{}".format(args.gpu_id)))



  5%|▌         | 1/20 [00:35<11:12, 35.39s/it]

Epoch: 0, epoch time: 0.519 min, tot time: 0.000day, L_train: 2.980, L_base: 3.906, L_test: 3.341, gap_train(%): -100.000, update: True
Epoch: 1, epoch time: 0.383 min, tot time: 0.001day, L_train: 2.914, L_base: 2.984, L_test: 3.237, gap_train(%): -100.000, update: True


 15%|█▌        | 3/20 [01:31<08:27, 29.84s/it]

Epoch: 2, epoch time: 0.385 min, tot time: 0.001day, L_train: 2.908, L_base: 2.912, L_test: 3.184, gap_train(%): -100.000, update: True
